### IMPORTS

In [1]:
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression

In [15]:
import tensorflow as tf
from tensorflow.python.keras import Sequential, Input, Model
from tensorflow.python.keras.layers import Dense, LSTM, Dropout

In [16]:
import ipynb.fs.full.splitting as splitting

In [17]:
# from tcn import TCN, tcn_full_summary

### LINEAR REGRESSION

In [6]:
def linear_regression(data, settings):
    
    # INSTANTIATE MODEL
    linear = LinearRegression()
    
    # TRAIN WITH TRAIN DATA
    model = linear.fit(data['train']['features'], data['train']['labels'])
    
    # PREDICT USING TEST DATA
    predictions = model.predict(data['test']['features'])
    
    return np.ndarray.flatten(predictions)

### LONG-SHORT TERM MEMORY

In [7]:
def add_lstm_layer(model, data, index, name, settings, shape):
    
    # AVAILABLE LSTM LAYERS
    available = {
        'lstm': LSTM,
        'dropout': Dropout,
        'dense': Dense
    }
    
    # SELECT THE CORRECT FUNCTION
    func = available[name]
    
    # IF AN ACTIVATION IS FOUND & THIS IS THE FIRST LAYER
    if 'activation' in settings and index == 0:
        model.add(func(
            settings['value'],
            input_shape=(shape[1], shape[2])
        ))
    
    # JUST AN ACTIVATION FUNCTION
    elif 'activation' in settings:
        model.add(func(
            settings['value'],
            activation=settings['activation']
        ))
        
    # OTHERWISE, DEFAULT TO JUST USING THE VALUE
    else:
        model.add(func(
            settings['value']
        ))

In [8]:
def long_short_term(data, settings):
    
    # INSTANTIATE MODEL
    model = Sequential()
    
    # TRAIN DATA GENERATOR
    train_generator = splitting.generator(
        data['train'],
        settings['morph'],
        shuffle=True
    )
    
    # LOOP THROUGH REQUESTED MODEL LAYERS
    for index, layer in enumerate(settings['layers']):
        
        # LAYER PROPS
        name = list(layer)[0]
        params = layer[name]
        
        # GENERATE & ADD THE LAYER
        add_lstm_layer(model, data, index, name, params, train_generator[0][0].shape)
    
    # COMPILE THE MODEL
    model.compile(
        loss=settings['loss'],
        optimizer=settings['optimizer']
    )
    
    # FIT USING THE TRAIN GENERATOR
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=settings['epochs'],
        verbose=0
    )
    
    # TEST DATA GENERATOR
    test_generator = splitting.generator(
        data['test'],
        settings['morph'],
        shuffle=False
    )
    
    # PREDICT USING TEST DATA
    predictions = model.predict(test_generator)
    
    return np.ndarray.flatten(predictions)

### TEMPORAL CONVOLUTIONAL NETWORK

In [9]:
def add_base_layer_to_tcn(name, model_output, settings, index):
    """Support function that adds Keras Layers to TCN model."""
    # AVAILABLE LAYERS
    available = {
        'dropout': Dropout,
        'dense': Dense
    }

    # SELECT THE CORRECT FUNCTION
    func = available[name]

    if name == 'dropout':
        return func(settings['value'])(model_output)

    else:
        return func(settings['value'])(model_output)

In [10]:
def add_tcn_layer(model_input, settings):
    """Support function that adds TCN Layer and requested Keras Layers to the TCN model"""

    # STARTING LAYER (TCN)
    tcn_string = ''
    try:
        for index, stats in enumerate(settings['layers'][0]['tcn']):
            # LAYER PROPS
            value = settings['layers'][0]['tcn'][stats]
            if list(settings['layers'][0]['tcn'])[-1]:
                tcn_string += str(stats) + '=' + str(value)
            else:
                tcn_string += str(stats) + '=' + str(value) + ','
        model_output = TCN(tcn_string)(model_input)

    except ValueError:
        model_output = TCN(return_sequences=False)(model_input)

    for index, layer in enumerate(settings['layers']):

        # LAYER PROPS
        name = list(layer)[0]
        params = layer[name]

        if index == 0:
            continue
        else:
            model_output = add_base_layer_to_tcn(name, model_output, params, index)

    return model_output

In [11]:
def temporal_convolutional_network(data, settings):

    # DECONSTRUCT VARS
    x_train = data['train']['features']
    y_train = data['train']['labels']
    x_validation = data['validation']['features']
    y_validation = data['validation']['labels']
    x_test = data['test']['features']
    scaler = data['scaler']
    timesteps = x_train.shape[0]
    input_dim_x = x_train.shape[1]
    input_dim_y = x_train.shape[2]
    batch_size = settings['batch']

    # INSTANTIATE KERAS TENSOR WITH Input()
    model_input = Input(shape=(input_dim_x, input_dim_y))

    #  INSTANTIATE MODEL LAYERS
    model_output = add_tcn_layer(model_input, settings)

    #  INSTANTIATE MODEL AND ASSIGN INPUT AND OUTPUT
    model = Model(inputs=[model_input], outputs=[model_output])

    # COMPILE THE MODEL
    model.compile(
        optimizer=settings['optimizer'],
        loss=settings['loss']
    )

    #  PRINT MODEL STATS
    tcn_full_summary(model, expand_residual_blocks=False)

    #  TRAIN THE MODEL WITH VALIDATION
    model.fit(x_train, y_train, epochs=settings['epochs'], validation_data=(x_validation, y_validation))

    # PREDICT USING TEST DATA
    predictions = model.predict(x_test)

    return {
        'model': model,
        'predictions': predictions
    }

### START TRAINING A MODEL

In [12]:
def start(dataset, name, settings):
    
    # AVAILABLE MODELS
    available = {
        'linreg': linear_regression,
        'lstm': long_short_term
    }
    
    # SELECT THE CORRECT FUNCTION & START
    return available[name](dataset, settings)